In [19]:
import numpy as np
import astropy
from astropy.table import QTable
from astroplan import Observer, FixedTarget
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz
from astroplan import download_IERS_A

In [20]:
download_IERS_A()

In [21]:
loud_galaxies = QTable.read('01_data.csv', format = 'ascii.csv' )
loud_galaxies

iau_name,alt_name,ra,dec,class
str9,str16,str10,str9,str14
0454-8105,PKS 0454-81,04 50 05.0,-81 01 02,QSO RADIO LOUD
0621-7841,PKS 0621-786,06 18 30.3,-78 43 02,QSO RADIO LOUD
0858-7707,PKS 0858-77,08 57 42.7,-77 19 32,QSO RADIO LOUD
0312-7703,PKS 0312-77,03 11 54.7,-76 51 52,QSO RADIO LOUD
0202-7634,PKS 0202-76,02 02 13.0,-76 20 06,QSO RADIO LOUD
0842-7529,PKS 0842-75,08 41 27.5,-75 40 26,QSO RADIO LOUD
0637-7513,PKS 0637-75,06 35 46.5,-75 16 17,QSO RADIO LOUD
2300-6823,PKS 2300-683,23 03 43.6,-68 07 37,QSO RADIO LOUD
2111-6806,1E 2111-681,21 15 44.1,-67 54 04,QSO RADIO LOUD


In [16]:
kitt = Observer.at_site('Kitt Peak')

In [17]:
kitt

<Observer: name='Kitt Peak',
    location (lon, lat, el)=(-111.6 deg, 31.963333333333342 deg, 2119.999999997953 m),
    timezone=<UTC>>